In [ ]:
import json
import math
import os
import glob
import time
import random
import warnings
from datetime import datetime
from collections import defaultdict

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as mpl_cm
import matplotlib.lines as mlines
from matplotlib.collections import LineCollection
from matplotlib.patches import Polygon as MplPolygon
from shapely.geometry import LineString, Point
from shapely.strtree import STRtree
from shapely.ops import unary_union
from scipy.spatial import cKDTree
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')

# ================================================================
# ПАРАМЕТРЫ
# ================================================================
SNAP_PRECISION     = 5
BRIDGE_RADIUS_M    = 500
TJUNCTION_RADIUS_M = 200
BUFFER_M           = 50
MEAN_LAT           = 61.0
COS_LAT            = math.cos(math.radians(MEAN_LAT))
DEG_TO_M           = 111_000
COORD_PRECISION    = 7
N_ROUNDS           = 3
STATION_SNAP_M     = 500
N_SELECT           = 10

node_counter = 0

# ================================================================
# ВВОД: ПУТЬ К ПАПКЕ С ДОРОЖНЫМИ ФАЙЛАМИ
# ================================================================
ROADS_DIR = input("Введите путь к папке с .json/.geojson файлами дорог: ").strip().strip('"').strip("'")

if not os.path.isdir(ROADS_DIR):
    raise FileNotFoundError(f"Папка не найдена: {ROADS_DIR}")

road_files = sorted(
    glob.glob(os.path.join(ROADS_DIR, '*.geojson')) +
    glob.glob(os.path.join(ROADS_DIR, '*.json'))
)
if not road_files:
    raise FileNotFoundError(f"В папке нет .json/.geojson файлов: {ROADS_DIR}")

print(f"Найдено файлов: {len(road_files)}")
for f in road_files:
    print(f"  - {os.path.basename(f)}")

# ================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ================================================================
def snap_coord(coord, prec=SNAP_PRECISION):
    return (round(coord[0], prec), round(coord[1], prec))

def make_nid(sc, prec=SNAP_PRECISION):
    return f"n_{sc[0]:.{prec}f}_{sc[1]:.{prec}f}"

def deg_to_m(lon, lat):
    return lon * DEG_TO_M * COS_LAT, lat * DEG_TO_M

def edge_length_m(c1, c2):
    dx = (c1[0] - c2[0]) * DEG_TO_M * COS_LAT
    dy = (c1[1] - c2[1]) * DEG_TO_M
    return math.hypot(dx, dy)

def linestring_length_m(coords):
    total = 0.0
    for i in range(len(coords) - 1):
        total += edge_length_m(coords[i], coords[i + 1])
    return total

def in_bbox(lon, lat):
    if BBOX is None:
        return True
    return BBOX_W <= lon <= BBOX_E and BBOX_S <= lat <= BBOX_N

class UF:
    def __init__(self, n):
        self.p = list(range(n))
        self.r = [0] * n
    def find(self, x):
        while self.p[x] != x:
            self.p[x] = self.p[self.p[x]]
            x = self.p[x]
        return x
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py: return False
        if self.r[px] < self.r[py]: px, py = py, px
        self.p[py] = px
        if self.r[px] == self.r[py]: self.r[px] += 1
        return True

def build_comp_map(G):
    comps = list(nx.connected_components(G))
    m = {}
    for ci, c in enumerate(comps):
        for n in c:
            m[n] = ci
    return m, comps

def graph_coords_m(G):
    nodes = list(G.nodes())
    arr = np.array([deg_to_m(G.nodes[n]['x'], G.nodes[n]['y']) for n in nodes])
    return nodes, arr

# ================================================================
# ЗАГРУЗКА GEOJSON
# ================================================================
def load_geojson_as_edges(filepath):
    global node_counter
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)

    G_temp = nx.Graph()
    source_file = os.path.basename(filepath)

    if data.get("type") != "FeatureCollection" or "features" not in data:
        print(f"  Пропущен (неподдерживаемый формат): {source_file}")
        return G_temp

    for feature in data["features"]:
        geom = feature.get("geometry", {})
        props = feature.get("properties", {})
        geom_type = geom.get("type")

        def process_linestring(coords):
            global node_counter
            if len(coords) < 2:
                return
            coords_r = [(round(lon, COORD_PRECISION), round(lat, COORD_PRECISION))
                        for lon, lat in coords]
            if coords_r[0] == coords_r[-1]:
                return
            sn = f"node_{node_counter}"; node_counter += 1
            en = f"node_{node_counter}"; node_counter += 1
            G_temp.add_node(sn, x=coords_r[0][0], y=coords_r[0][1])
            G_temp.add_node(en, x=coords_r[-1][0], y=coords_r[-1][1])
            G_temp.add_edge(sn, en,
                           length_m=round(linestring_length_m(coords_r), 2),
                           road_type=props.get("highway", props.get("type", "")),
                           source_file=source_file,
                           geometry=coords_r,
                           original_edge=True)

        if geom_type == "LineString":
            process_linestring(geom["coordinates"])
        elif geom_type == "MultiLineString":
            for line_coords in geom["coordinates"]:
                process_linestring(line_coords)

    return G_temp

# ================================================================
# ОПЕРАЦИИ СШИВКИ
# ================================================================
def snap_graph(G):
    snap_map = defaultdict(list)
    for node in G.nodes():
        snapped = snap_coord((G.nodes[node]['x'], G.nodes[node]['y']))
        snap_map[snapped].append(node)

    newG = nx.Graph()
    old_to_new = {}
    for snapped, nodes in snap_map.items():
        new_node = make_nid(snapped)
        newG.add_node(new_node, x=snapped[0], y=snapped[1])
        for old_node in nodes:
            old_to_new[old_node] = new_node

    for u, v, d in G.edges(data=True):
        nu, nv = old_to_new[u], old_to_new[v]
        if nu == nv:
            continue
        if not newG.has_edge(nu, nv):
            if 'geometry' in d:
                geom = d['geometry'].copy()
                geom[0] = (newG.nodes[nu]['x'], newG.nodes[nu]['y'])
                geom[-1] = (newG.nodes[nv]['x'], newG.nodes[nv]['y'])
                d = dict(d)
                d['geometry'] = geom
            newG.add_edge(nu, nv, **d)

    return newG


def do_bridge_stitching(G, label=''):
    total = 0
    while True:
        nl, coords_m = graph_coords_m(G)
        n2i = {n: i for i, n in enumerate(nl)}
        tree = cKDTree(coords_m)
        pairs = tree.query_pairs(BRIDGE_RADIUS_M)

        uf = UF(len(nl))
        for u, v in G.edges():
            uf.union(n2i[u], n2i[v])

        cross = sorted([(np.linalg.norm(coords_m[i]-coords_m[j]), i, j)
                        for i, j in pairs if uf.find(i) != uf.find(j)])

        added = 0
        for d_val, i, j in cross:
            if uf.find(i) != uf.find(j):
                uf.union(i, j)
                G.add_edge(nl[i], nl[j],
                           length_m=float(d_val),
                           road_type='bridge',
                           source_file=f'auto_bridge{label}',
                           original_edge=False)
                added += 1
        total += added
        if added == 0:
            break
    return total


def do_edge_intersections(G):
    edges_info = []
    edge_lines = []
    for u, v, d in G.edges(data=True):
        if 'geometry' in d and d['geometry']:
            coords = d['geometry']
            line = LineString(coords)
        else:
            p1 = (G.nodes[u]['x'], G.nodes[u]['y'])
            p2 = (G.nodes[v]['x'], G.nodes[v]['y'])
            line = LineString([p1, p2])
            coords = [p1, p2]
        edges_info.append((u, v, dict(d), coords))
        edge_lines.append(line)

    strtree_ei = STRtree(edge_lines)
    split_pts = defaultdict(list)
    eps = 10 ** -(SNAP_PRECISION + 1)

    for i, (u1, v1, d1, coords1) in enumerate(edges_info):
        for j in strtree_ei.query(edge_lines[i]):
            if j <= i: continue
            u2, v2, d2, coords2 = edges_info[j]
            if u1 in (u2, v2) or v1 in (u2, v2):
                continue
            if not edge_lines[i].intersects(edge_lines[j]):
                continue
            pt = edge_lines[i].intersection(edge_lines[j])
            if pt.geom_type != 'Point':
                continue
            endpoints = [coords1[0], coords1[-1], coords2[0], coords2[-1]]
            if any(abs(pt.x-px)<eps and abs(pt.y-py)<eps for px, py in endpoints):
                continue
            sc = snap_coord((pt.x, pt.y))
            split_pts[i].append(sc)
            split_pts[j].append(sc)

    to_remove = set()
    to_add = []
    for ei, pts in split_pts.items():
        u, v, d, coords = edges_info[ei]
        to_remove.add((u, v))
        all_pts = [coords[0]] + pts + [coords[-1]]
        sx, sy = coords[0]
        all_pts.sort(key=lambda c: (c[0]-sx)**2 + (c[1]-sy)**2)
        uniq = [all_pts[0]]
        for p in all_pts[1:]:
            if p != uniq[-1]:
                uniq.append(p)
        for k in range(len(uniq)-1):
            to_add.append((uniq[k], uniq[k+1],
                           d.get('road_type', ''),
                           d.get('source_file', '')))

    for u, v in to_remove:
        if G.has_edge(u, v):
            G.remove_edge(u, v)
    for sc, ec, rt, sf in to_add:
        if sc == ec: continue
        sn, en = make_nid(sc), make_nid(ec)
        if sn not in G: G.add_node(sn, x=sc[0], y=sc[1])
        if en not in G: G.add_node(en, x=ec[0], y=ec[1])
        G.add_edge(sn, en, length_m=edge_length_m(sc, ec),
                   road_type=rt, source_file=sf, original_edge=False)
    G.remove_nodes_from(list(nx.isolates(G)))


def do_tjunction(G):
    comp_map, _ = build_comp_map(G)
    dangling = [n for n in G.nodes() if G.degree(n) == 1]

    tj_edges = []
    tj_geoms = []
    for u, v, d in G.edges(data=True):
        if 'geometry' in d and d['geometry']:
            line = LineString(d['geometry'])
        else:
            line = LineString([(G.nodes[u]['x'], G.nodes[u]['y']),
                               (G.nodes[v]['x'], G.nodes[v]['y'])])
        tj_edges.append((u, v, d))
        tj_geoms.append(line)

    tj_tree = STRtree(tj_geoms)
    tj_rad_deg = TJUNCTION_RADIUS_M / DEG_TO_M
    edge_projs = defaultdict(list)

    for n in dangling:
        nd = G.nodes[n]
        pt = Point(nd['x'], nd['y'])
        cn = comp_map[n]
        buf = pt.buffer(tj_rad_deg)
        best_dist = TJUNCTION_RADIUS_M
        best_idx = None
        best_proj = None
        for idx in tj_tree.query(buf):
            eu, ev, ed = tj_edges[idx]
            if comp_map.get(eu) == cn: continue
            proj = tj_geoms[idx].interpolate(tj_geoms[idx].project(pt))
            dm = edge_length_m((pt.x, pt.y), (proj.x, proj.y))
            if dm < best_dist:
                best_dist = dm
                best_idx = idx
                best_proj = snap_coord((proj.x, proj.y))
        if best_idx is not None:
            edge_projs[best_idx].append((best_proj, n, best_dist))

    for ei, projs in edge_projs.items():
        u, v, d = tj_edges[ei]
        if not G.has_edge(u, v): continue
        rt = d.get('road_type', '')
        sf = d.get('source_file', '')
        if 'geometry' in d and d['geometry']:
            coords = d['geometry']
        else:
            coords = [(G.nodes[u]['x'], G.nodes[u]['y']),
                      (G.nodes[v]['x'], G.nodes[v]['y'])]
        start, end = snap_coord(coords[0]), snap_coord(coords[-1])
        all_p = [start]
        pm = defaultdict(list)
        for proj_c, dn, dm in projs:
            if proj_c != start and proj_c != end:
                all_p.append(proj_c)
            pm[proj_c].append((dn, dm))
        all_p.append(end)
        sx, sy = start
        all_p.sort(key=lambda c: (c[0]-sx)**2+(c[1]-sy)**2)
        uniq = [all_p[0]]
        for p in all_p[1:]:
            if p != uniq[-1]: uniq.append(p)
        G.remove_edge(u, v)
        for k in range(len(uniq)-1):
            sc, ec = uniq[k], uniq[k+1]
            if sc == ec: continue
            sn, en = make_nid(sc), make_nid(ec)
            if sn not in G: G.add_node(sn, x=sc[0], y=sc[1])
            if en not in G: G.add_node(en, x=ec[0], y=ec[1])
            G.add_edge(sn, en, length_m=edge_length_m(sc, ec),
                       road_type=rt, source_file=sf, original_edge=False)
        for proj_c, dang_list in pm.items():
            pid = make_nid(proj_c)
            if pid not in G: G.add_node(pid, x=proj_c[0], y=proj_c[1])
            for dn, dm in dang_list:
                G.add_edge(dn, pid, length_m=dm,
                           road_type='t_junction',
                           source_file='auto_tj',
                           original_edge=False)
    G.remove_nodes_from(list(nx.isolates(G)))


def do_buffer_merge(G):
    comp_map2, comps2 = build_comp_map(G)
    buf_deg = BUFFER_M / DEG_TO_M
    comp_edges_geom = defaultdict(list)
    for u, v, d in G.edges(data=True):
        ci = comp_map2[u]
        if 'geometry' in d and d['geometry']:
            line = LineString(d['geometry'])
        else:
            line = LineString([(G.nodes[u]['x'], G.nodes[u]['y']),
                               (G.nodes[v]['x'], G.nodes[v]['y'])])
        comp_edges_geom[ci].append(line)

    big_comps = [(ci, geoms) for ci, geoms in comp_edges_geom.items() if len(geoms) >= 3]
    comp_buffers = []
    comp_ids_buf = []
    for ci, geoms in big_comps:
        merged = unary_union(geoms).buffer(buf_deg)
        comp_buffers.append(merged)
        comp_ids_buf.append(ci)

    buf_tree = STRtree(comp_buffers)
    buf_pairs = set()
    for i, geom in enumerate(comp_buffers):
        for j in buf_tree.query(geom):
            if j <= i: continue
            ci, cj = comp_ids_buf[i], comp_ids_buf[j]
            if ci == cj: continue
            if comp_buffers[i].intersects(comp_buffers[j]):
                buf_pairs.add((i, j))

    for i, j in buf_pairs:
        ci, cj = comp_ids_buf[i], comp_ids_buf[j]
        nodes_i = list(comps2[ci])
        nodes_j = list(comps2[cj])
        if not nodes_i or not nodes_j: continue
        coords_i = np.array([deg_to_m(G.nodes[n]['x'], G.nodes[n]['y']) for n in nodes_i])
        coords_j = np.array([deg_to_m(G.nodes[n]['x'], G.nodes[n]['y']) for n in nodes_j])
        ti = cKDTree(coords_i)
        dists, idxs = ti.query(coords_j, k=1)
        best = np.argmin(dists)
        bi = idxs[best]
        dm = dists[best]
        if dm > BRIDGE_RADIUS_M: continue
        ni, nj = nodes_i[bi], nodes_j[best]
        if not G.has_edge(ni, nj):
            G.add_edge(ni, nj, length_m=float(dm),
                       road_type='buffer_merge',
                       source_file='auto_buffer',
                       original_edge=False)
    G.remove_nodes_from(list(nx.isolates(G)))

# ================================================================
# КЛАССИФИКАЦИЯ ФОНОВЫХ ОБЪЕКТОВ
# ================================================================
WATER_KEYS = ('water', 'river', 'lake', 'sea', 'ocean', 'bay', 'strait',
              'reservoir', 'pond', 'вод', 'река', 'озер', 'море', 'залив',
              'пруд', 'водохр', 'ручей', 'канал')
COAST_KEYS = ('coastline', 'coast', 'берег', 'побереж')
RAIL_KEYS  = ('railway', 'rail', 'жд', 'жел_дор', 'железн')

def classify_feature(feat, filename):
    props = feat.get('properties', {})
    vals = ' '.join(str(v).lower() for v in props.values() if v)
    keys_present = set(k.lower() for k in props.keys())
    fn = filename.lower()

    if 'railway' in keys_present:
        return 'railway'
    if any(k in fn for k in RAIL_KEYS):
        return 'railway'
    if any(k in vals for k in ('rail', 'railway')):
        return 'railway'

    nat = str(props.get('natural', '')).lower()
    if nat == 'coastline':
        return 'coastline'
    if any(k in fn for k in COAST_KEYS):
        return 'coastline'
    if any(k in vals for k in COAST_KEYS):
        return 'coastline'

    if 'waterway' in keys_present or 'water' in keys_present:
        return 'water'
    if nat in ('water', 'bay', 'strait', 'wetland'):
        return 'water'
    if any(k in fn for k in WATER_KEYS):
        return 'water'
    if any(k in vals for k in WATER_KEYS):
        return 'water'

    return 'other'

BG_STYLES = {
    'water':     {'line_color': '#2196F3', 'fill_color': '#BBDEFB', 'edge_color': '#1976D2',
                  'lw': 0.6, 'alpha': 0.6, 'fill_alpha': 0.35},
    'coastline': {'line_color': '#2196F3', 'fill_color': '#BBDEFB', 'edge_color': '#1976D2',
                  'lw': 0.8, 'alpha': 0.7, 'fill_alpha': 0.1},
    'railway':   {'line_color': '#000000', 'fill_color': '#000000', 'edge_color': '#000000',
                  'lw': 1.2, 'alpha': 0.8, 'fill_alpha': 0.3},
    'other':     {'line_color': '#aaaaaa', 'fill_color': '#e0e0e0', 'edge_color': '#bbbbbb',
                  'lw': 0.4, 'alpha': 0.5, 'fill_alpha': 0.4},
}

# Цвета маршрутов — исключаем синий, чёрный и серый
ROUTE_COLORS = [
    '#E53935',  # красный
    '#43A047',  # зелёный
    '#FB8C00',  # оранжевый
    '#8E24AA',  # фиолетовый
    '#00ACC1',  # бирюзовый
    '#F4511E',  # тёмно-оранжевый
    '#7CB342',  # лайм
    '#D81B60',  # малиновый
    '#FFB300',  # янтарный
    '#5E35B1',  # индиго
]

# ================================================================
# 1. ЗАГРУЗКА ФАЙЛОВ
# ================================================================
print("=" * 72)
print("[1/8] Загрузка файлов дорог...")
graphs = []
for filepath in tqdm(road_files, desc="Загрузка"):
    G_temp = load_geojson_as_edges(filepath)
    if G_temp.number_of_edges() > 0:
        graphs.append(G_temp)

G = nx.Graph()
for G_temp in graphs:
    for node, attrs in G_temp.nodes(data=True):
        G.add_node(node, **attrs)
    for u, v, attrs in G_temp.edges(data=True):
        G.add_edge(u, v, **attrs)

print(f"  Узлов: {G.number_of_nodes():,}  |  Рёбер: {G.number_of_edges():,}  |  "
      f"Компонент: {nx.number_connected_components(G):,}")

# ================================================================
# ФИЛЬТРАЦИЯ ПО КООРДИНАТАМ (перед сшивкой)
# ================================================================
print("\n[1.5/8] Ограничения по координатам...")
use_bbox = input("Ограничить область по координатам? (да/нет): ").strip().lower()

BBOX = None
BBOX_N = BBOX_S = BBOX_W = BBOX_E = None

if use_bbox in ('да', 'yes', 'y', 'д'):
    BBOX_N = float(input("  Северная граница (широта, напр. 61.50): "))
    BBOX_S = float(input("  Южная граница   (широта, напр. 60.00): "))
    BBOX_W = float(input("  Западная граница (долгота, напр. 29.00): "))
    BBOX_E = float(input("  Восточная граница (долгота, напр. 32.00): "))
    BBOX = (BBOX_S, BBOX_N, BBOX_W, BBOX_E)

    n_before = G.number_of_nodes()
    to_remove = [n for n in G.nodes()
                 if not (BBOX_W <= G.nodes[n]['x'] <= BBOX_E and
                         BBOX_S <= G.nodes[n]['y'] <= BBOX_N)]
    G.remove_nodes_from(to_remove)
    G.remove_nodes_from(list(nx.isolates(G)))
    print(f"  Удалено узлов вне области: {len(to_remove):,}")
    print(f"  После фильтрации: Узлов: {G.number_of_nodes():,}  |  "
          f"Рёбер: {G.number_of_edges():,}  |  "
          f"Компонент: {nx.number_connected_components(G):,}")
    print(f"  Область: Ш {BBOX_S}°–{BBOX_N}°, Д {BBOX_W}°–{BBOX_E}°")
else:
    print("  Ограничения не заданы — используется весь граф.")

# ================================================================
# 2. СШИВКА ГРАФА (каждый метод — свой прогресс-бар)
# ================================================================
print("\n[2/8] Сшивка графа")
print("=" * 72)

def log_stats(name, G, t0):
    dt = time.time() - t0
    print(f"      Узлов: {G.number_of_nodes():,}  |  Рёбер: {G.number_of_edges():,}  |  "
          f"Компонент: {nx.number_connected_components(G):,}  |  {dt:.1f} сек")

# --- SNAP ---
print("  [SNAP] Привязка координат...")
pbar_snap = tqdm(total=1, desc="  SNAP", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
t0 = time.time()
G = snap_graph(G)
pbar_snap.update(1)
pbar_snap.close()
log_stats("SNAP", G, t0)

# --- МОСТЫ (первичные) ---
print("  [BRIDGE] Первичная сшивка мостами...")
pbar_br = tqdm(total=1, desc="  BRIDGE", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
t0 = time.time()
do_bridge_stitching(G)
pbar_br.update(1)
pbar_br.close()
log_stats("BRIDGE", G, t0)

# --- РАУНДЫ ---
for r in range(1, N_ROUNDS + 1):
    print(f"\n  --- Раунд {r}/{N_ROUNDS} ---")

    print(f"  [INTERSECT {r}] Пересечения рёбер...")
    pbar_int = tqdm(total=1, desc=f"  INTERSECT {r}", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    t0 = time.time()
    do_edge_intersections(G)
    pbar_int.update(1)
    pbar_int.close()
    log_stats(f"INTERSECT {r}", G, t0)

    print(f"  [T-JUNCTION {r}] Привязка висячих вершин...")
    pbar_tj = tqdm(total=1, desc=f"  T-JUNCTION {r}", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    t0 = time.time()
    do_tjunction(G)
    pbar_tj.update(1)
    pbar_tj.close()
    log_stats(f"T-JUNCTION {r}", G, t0)

    print(f"  [BUFFER {r}] Буферное слияние...")
    pbar_buf = tqdm(total=1, desc=f"  BUFFER {r}", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    t0 = time.time()
    do_buffer_merge(G)
    pbar_buf.update(1)
    pbar_buf.close()
    log_stats(f"BUFFER {r}", G, t0)

    print(f"  [BRIDGE {r}] Дополнительная сшивка мостами...")
    pbar_br2 = tqdm(total=1, desc=f"  BRIDGE {r}", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')
    t0 = time.time()
    do_bridge_stitching(G, label='_r')
    pbar_br2.update(1)
    pbar_br2.close()
    log_stats(f"BRIDGE {r}", G, t0)

print("=" * 72)
print(f"  ИТОГ:  Узлов: {G.number_of_nodes():,}  |  "
      f"Рёбер: {G.number_of_edges():,}  |  "
      f"Компонент: {nx.number_connected_components(G):,}")

# ================================================================
# 3. ВИЗУАЛИЗАЦИЯ ТОП-10 КОМПОНЕНТ
# ================================================================
print("\n[3/8] Визуализация топ-10 компонент...")

components = sorted(nx.connected_components(G), key=len, reverse=True)
node_comp = {}
for ci, c in enumerate(components):
    for n in c:
        node_comp[n] = ci

show_n = min(10, len(components))
comp_segs = [[] for _ in range(show_n)]
other_segs = []

for u, v, d in G.edges(data=True):
    ci = node_comp[u]
    if 'geometry' in d and d['geometry']:
        coords = d['geometry']
        for i in range(len(coords) - 1):
            seg = [coords[i], coords[i + 1]]
            if ci < show_n:
                comp_segs[ci].append(seg)
            else:
                other_segs.append(seg)
    else:
        p1 = (G.nodes[u]['x'], G.nodes[u]['y'])
        p2 = (G.nodes[v]['x'], G.nodes[v]['y'])
        if ci < show_n:
            comp_segs[ci].append([p1, p2])
        else:
            other_segs.append([p1, p2])

cmap_c = mpl_cm.get_cmap('tab10', max(show_n, 1))
fig_comp, ax_comp = plt.subplots(figsize=(16, 12))

if other_segs:
    ax_comp.add_collection(LineCollection(other_segs, colors='#dddddd',
                                          linewidths=0.15, alpha=0.3, label='остальные'))

for i in range(show_n - 1, -1, -1):
    if not comp_segs[i]: continue
    c = cmap_c(i)
    lw = 1.5 if i == 0 else 0.6
    ax_comp.add_collection(LineCollection(comp_segs[i], colors=[c], linewidths=lw, alpha=0.85,
                                          label=f'#{i+1} ({len(components[i]):,} узл.)'))

ax_comp.autoscale()
ax_comp.set_aspect('equal')
ax_comp.set_xlabel('Долгота')
ax_comp.set_ylabel('Широта')
bbox_info = (f" | Область: Ш {BBOX_S}°–{BBOX_N}°, Д {BBOX_W}°–{BBOX_E}°"
             if BBOX is not None else "")
ax_comp.set_title(f'Топ-10 компонент дорожного графа{bbox_info}\n'
                  f'Компонент: {len(components):,} | '
                  f'Узлов: {G.number_of_nodes():,} | '
                  f'Рёбер: {G.number_of_edges():,}', fontsize=12)
ax_comp.legend(loc='upper left', fontsize=8)
ax_comp.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

# ================================================================
# 4. ПРОДОЛЖИТЬ?
# ================================================================
cont = input("\nПродолжить работу? (да/нет): ").strip().lower()
if cont not in ('да', 'yes', 'y', 'д'):
    print("Работа завершена.")
    raise SystemExit

# ================================================================
# 5. ЗАГРУЗКА И ПРОЕЦИРОВАНИЕ СТАНЦИЙ
# ================================================================
STATIONS_PATH = input("Путь к файлу ЖД-станций (GeoJSON): ").strip().strip('"').strip("'")

print("\n[4/8] Загрузка ЖД-станций...")
with open(STATIONS_PATH, 'r', encoding='utf-8') as f:
    sdata = json.load(f)

stations_raw = []
skipped_bbox = 0
for feat in sdata['features']:
    geom = feat['geometry']
    props = feat.get('properties', {})
    if geom['type'] == 'Point':
        lon, lat = geom['coordinates'][:2]
        if not in_bbox(lon, lat):
            skipped_bbox += 1
            continue
        name = (props.get('name') or props.get('NAME') or
                props.get('title') or props.get('TITLE') or
                f'Станция_{len(stations_raw)+1}')
        stations_raw.append({'name': str(name), 'lon': lon, 'lat': lat})

print(f"  Найдено станций: {len(stations_raw)}"
      + (f"  (вне области: {skipped_bbox})" if skipped_bbox > 0 else ""))

print("\n[5/8] Проецирование станций на граф...")

edges_data = []
edge_lines = []
for u, v, d in G.edges(data=True):
    if 'geometry' in d and d['geometry']:
        line = LineString(d['geometry'])
        coords = d['geometry']
    else:
        c1 = (G.nodes[u]['x'], G.nodes[u]['y'])
        c2 = (G.nodes[v]['x'], G.nodes[v]['y'])
        line = LineString([c1, c2])
        coords = [c1, c2]
    edges_data.append((u, v, d, coords))
    edge_lines.append(line)

strtree = STRtree(edge_lines)
snap_rad = STATION_SNAP_M / DEG_TO_M

stations = []
for st in tqdm(stations_raw, desc="Проецирование"):
    pt = Point(st['lon'], st['lat'])
    buf = pt.buffer(snap_rad)

    best_dist = float('inf')
    best_proj = None
    best_idx = None

    for idx in strtree.query(buf):
        line = edge_lines[idx]
        proj = line.interpolate(line.project(pt))
        dm = edge_length_m((pt.x, pt.y), (proj.x, proj.y))
        if dm < best_dist and dm <= STATION_SNAP_M:
            best_dist = dm
            best_proj = snap_coord((proj.x, proj.y))
            best_idx = idx

    if best_proj is not None:
        node_id = f"st_{len(stations)}"
        u, v, d, coords = edges_data[best_idx]

        G.add_node(node_id, x=best_proj[0], y=best_proj[1],
                   is_station=True, station_name=st['name'])

        du = edge_length_m(best_proj, (G.nodes[u]['x'], G.nodes[u]['y']))
        dv = edge_length_m(best_proj, (G.nodes[v]['x'], G.nodes[v]['y']))
        G.add_edge(node_id, u, length_m=du, road_type='station_link')
        G.add_edge(node_id, v, length_m=dv, road_type='station_link')

        stations.append({
            'name': st['name'], 'node_id': node_id,
            'lon': best_proj[0], 'lat': best_proj[1],
            'orig_lon': st['lon'], 'orig_lat': st['lat'],
            'snap_dist_m': round(best_dist, 1)
        })
    else:
        print(f"  ! '{st['name']}' — нет рёбер в радиусе {STATION_SNAP_M} м")

print(f"  Спроецировано: {len(stations)} / {len(stations_raw)}")

# ================================================================
# 6. ВЫБОР СТАНЦИЙ И ПОСТРОЕНИЕ МАРШРУТОВ
# ================================================================
MIN_ROUTE_KM = float(input("Мин. длина маршрута (км): "))
MAX_ROUTE_KM = float(input("Макс. длина маршрута (км): "))

print(f"\n[6/8] Построение маршрутов ({MIN_ROUTE_KM}–{MAX_ROUTE_KM} км)...")

n_sel = min(N_SELECT, len(stations))
selected = random.sample(stations, n_sel)
print(f"  Выбрано {n_sel} станций:")
for i, s in enumerate(selected, 1):
    print(f"    {i}. {s['name']}")

min_m = MIN_ROUTE_KM * 1000
max_m = MAX_ROUTE_KM * 1000

routes = []
for src in tqdm(selected, desc="Поиск маршрутов"):
    found = False
    for tgt in stations:
        if tgt['node_id'] == src['node_id']:
            continue
        try:
            path = nx.shortest_path(G, src['node_id'], tgt['node_id'], weight='length_m')
            length = sum(G[path[k]][path[k+1]]['length_m'] for k in range(len(path)-1))
            if min_m <= length <= max_m:
                routes.append({
                    'from': src['name'], 'to': tgt['name'],
                    'from_id': src['node_id'], 'to_id': tgt['node_id'],
                    'length_m': length, 'path': path
                })
                found = True
                break
        except nx.NetworkXNoPath:
            pass
    if not found:
        print(f"  ! Маршрут не найден для '{src['name']}'")

print(f"  Маршрутов: {len(routes)}")
if routes:
    routes.sort(key=lambda r: r['length_m'])
    print(f"  Длина: {routes[0]['length_m']/1000:.1f} — {routes[-1]['length_m']/1000:.1f} км")

# ================================================================
# ТАБЛИЦА МАРШРУТОВ
# ================================================================
table_header = f"  {'№':<4} {'Старт':<25} {'Финиш':<25} {'Длина, км':>10}"
table_sep = "-" * 72
table_lines = []
table_lines.append("=" * 72)
table_lines.append(table_header)
table_lines.append(table_sep)
for idx, r in enumerate(routes, 1):
    table_lines.append(f"  {idx:<4} {r['from']:<25} {r['to']:<25} {r['length_m']/1000:>10.1f}")
table_lines.append("=" * 72)
table_lines.append(f"  Итого маршрутов: {len(routes)}")
table_text = "\n".join(table_lines)
print("\n" + table_text)

# ================================================================
# 7. ЗАГРУЗКА ФОНОВЫХ ФАЙЛОВ
# ================================================================
bg_dir = input("\n[7/8] Путь к папке с фоновыми GeoJSON/JSON файлами: ").strip().strip('"').strip("'")
bg_classified = {'water': [], 'coastline': [], 'railway': [], 'other': []}
if os.path.isdir(bg_dir):
    bg_files = sorted(
        glob.glob(os.path.join(bg_dir, '*.geojson')) +
        glob.glob(os.path.join(bg_dir, '*.json'))
    )
    print(f"  Найдено файлов: {len(bg_files)}")
    counts = defaultdict(int)
    for bf in bg_files:
        fname = os.path.basename(bf)
        print(f"    - {fname}", end="")
        with open(bf, 'r', encoding='utf-8') as f:
            bdata = json.load(f)
        file_counts = defaultdict(int)
        for feat in bdata.get('features', []):
            cat = classify_feature(feat, fname)
            bg_classified[cat].append(feat)
            counts[cat] += 1
            file_counts[cat] += 1
        parts = [f"{cat}: {n}" for cat, n in sorted(file_counts.items())]
        print("  (" + ", ".join(parts) + ")")
    print(f"  Итого: вода={counts['water']}, берег={counts['coastline']}, "
          f"жд={counts['railway']}, прочее={counts['other']}")
else:
    print(f"  Папка не найдена: {bg_dir} — визуализация без фона")

# ================================================================
# 8. ВИЗУАЛИЗАЦИЯ
# ================================================================
print("\n[8/8] Визуализация...")

road_segs = []
for u, v, d in G.edges(data=True):
    if d.get('road_type') == 'station_link':
        continue
    if 'geometry' in d and d['geometry']:
        coords = d['geometry']
        for i in range(len(coords)-1):
            road_segs.append([coords[i], coords[i+1]])
    else:
        p1 = (G.nodes[u]['x'], G.nodes[u]['y'])
        p2 = (G.nodes[v]['x'], G.nodes[v]['y'])
        road_segs.append([p1, p2])

route_segs = defaultdict(list)
for route in routes:
    path = route['path']
    key = route['from_id']
    for k in range(len(path)-1):
        u, v = path[k], path[k+1]
        d = G[u][v]
        if 'geometry' in d and d['geometry']:
            coords = d['geometry']
            for j in range(len(coords)-1):
                route_segs[key].append([coords[j], coords[j+1]])
        else:
            p1 = (G.nodes[u]['x'], G.nodes[u]['y'])
            p2 = (G.nodes[v]['x'], G.nodes[v]['y'])
            route_segs[key].append([p1, p2])

fig, ax = plt.subplots(figsize=(144, 108))

# --- Фоновые объекты ---
def draw_bg(features, cat):
    s = BG_STYLES[cat]
    for feat in features:
        geom = feat.get('geometry', {})
        gt = geom.get('type', '')
        crd = geom.get('coordinates', [])
        if gt == 'LineString':
            xs = [p[0] for p in crd]
            ys = [p[1] for p in crd]
            ax.plot(xs, ys, color=s['line_color'], linewidth=s['lw'],
                    alpha=s['alpha'], zorder=1)
        elif gt == 'MultiLineString':
            for line in crd:
                xs = [p[0] for p in line]
                ys = [p[1] for p in line]
                ax.plot(xs, ys, color=s['line_color'], linewidth=s['lw'],
                        alpha=s['alpha'], zorder=1)
        elif gt == 'Polygon':
            ring = crd[0]
            poly = MplPolygon([(p[0], p[1]) for p in ring], closed=True,
                              fc=s['fill_color'], ec=s['edge_color'],
                              linewidth=s['lw'], alpha=s['fill_alpha'], zorder=1)
            ax.add_patch(poly)
        elif gt == 'MultiPolygon':
            for polygon in crd:
                ring = polygon[0]
                poly = MplPolygon([(p[0], p[1]) for p in ring], closed=True,
                                  fc=s['fill_color'], ec=s['edge_color'],
                                  linewidth=s['lw'], alpha=s['fill_alpha'], zorder=1)
                ax.add_patch(poly)
        elif gt == 'Point':
            ax.plot(crd[0], crd[1], '.', color=s['line_color'],
                    markersize=2, alpha=s['alpha'], zorder=1)

for cat in ('other', 'water', 'coastline', 'railway'):
    draw_bg(bg_classified[cat], cat)

# --- Дорожный граф ---
if road_segs:
    ax.add_collection(LineCollection(road_segs, colors='#dddddd',
                                     linewidths=0.3, alpha=0.4, zorder=2))

# --- Маршруты ---
route_color_map = {}
for i, src in enumerate(selected):
    segs = route_segs.get(src['node_id'], [])
    color = ROUTE_COLORS[i % len(ROUTE_COLORS)]
    route_color_map[src['node_id']] = color
    if segs:
        ax.add_collection(LineCollection(segs, colors=[color],
                                         linewidths=2.5, alpha=0.85, zorder=3))

# --- Станции ---
# Обычные станции — мелкий кружок, подпись 10pt
# Выбранные станции — красный треугольник вверх, подпись 14pt жирная
sel_ids = {s['node_id'] for s in selected}
for st in stations:
    is_sel = st['node_id'] in sel_ids
    if is_sel:
        ax.plot(st['lon'], st['lat'], '^',
                color='red', markersize=18,
                markeredgecolor='#8B0000', markeredgewidth=1.0,
                zorder=5)
        ax.annotate(st['name'], (st['lon'], st['lat']),
                    fontsize=14, fontweight='bold',
                    xytext=(6, 8), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.85, ec='#cccccc'),
                    zorder=6)
    else:
        ax.plot(st['lon'], st['lat'], 'o',
                color='#555555', markersize=6,
                zorder=5)
        ax.annotate(st['name'], (st['lon'], st['lat']),
                    fontsize=10,
                    xytext=(5, 5), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.2', fc='white', alpha=0.7, ec='none'),
                    zorder=6)

# --- Информация по маршрутам (справа наверху) ---
legend_handles = []
for route in routes:
    color = route_color_map.get(route['from_id'], '#E53935')
    km = route['length_m'] / 1000
    label = route['from'] + ' \u2014 ' + route['to'] + '  ' + f"{km:.1f}" + ' км'
    legend_handles.append(mlines.Line2D([], [], color=color, linewidth=2.5, label=label))

if legend_handles:
    ax.legend(handles=legend_handles, loc='upper right', fontsize=10,
              framealpha=0.85, edgecolor='#cccccc', title='Маршруты',
              title_fontsize=12)

# --- Границы отображения ---
if BBOX is not None:
    pad_lat = 10_000 / DEG_TO_M
    pad_lon = 10_000 / (DEG_TO_M * COS_LAT)
    ax.set_xlim(BBOX_W - pad_lon, BBOX_E + pad_lon)
    ax.set_ylim(BBOX_S - pad_lat, BBOX_N + pad_lat)
else:
    ax.autoscale()
ax.set_aspect('equal')
ax.set_xlabel('Долгота', fontsize=12)
ax.set_ylabel('Широта', fontsize=12)
title_bbox = (f" | Область: Ш {BBOX_S}°–{BBOX_N}°, Д {BBOX_W}°–{BBOX_E}°"
              if BBOX is not None else "")
ax.set_title(f'Маршруты от {n_sel} ЖД-станций{title_bbox}\n'
             f'Всего станций: {len(stations)} | '
             f'Маршрутов: {len(routes)} | '
             f'{MIN_ROUTE_KM}\u2013{MAX_ROUTE_KM} км', fontsize=14)
ax.grid(True, alpha=0.2)
plt.tight_layout()
plt.show()

# ================================================================
# 9. СОХРАНЕНИЕ
# ================================================================
save = input("\nСохранить результаты? (да/нет): ").strip().lower()
if save in ('да', 'yes', 'y', 'д'):
    SAVE_DIR = input("Путь к папке для сохранения: ").strip().strip('"').strip("'")
    if not os.path.isdir(SAVE_DIR):
        raise FileNotFoundError("Папка не найдена: " + SAVE_DIR)

    ts = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    out_dir = os.path.join(SAVE_DIR, ts)
    os.makedirs(out_dir, exist_ok=True)

    # PNG
    png_path = os.path.join(out_dir, f'roads_and_tracks_{ts}.png')
    fig.savefig(png_path, dpi=150, bbox_inches='tight')
    print(f"  PNG: {png_path}")

    # TXT
    txt_path = os.path.join(out_dir, f'roads_and_tracks_{ts}.txt')
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(table_text + "\n")
    print(f"  TXT: {txt_path}")

    # GeoJSON — карта со всеми объектами
    features_out = []

    # фоновые объекты
    for cat, feats in bg_classified.items():
        for feat in feats:
            feat_copy = dict(feat)
            if 'properties' not in feat_copy:
                feat_copy['properties'] = {}
            feat_copy['properties']['_bg_category'] = cat
            features_out.append(feat_copy)

    # станции
    for st in stations:
        features_out.append({
            'type': 'Feature',
            'geometry': {'type': 'Point',
                         'coordinates': [st['lon'], st['lat']]},
            'properties': {
                'name': st['name'],
                'type': 'station',
                'snap_dist_m': st['snap_dist_m'],
                'selected': st['node_id'] in sel_ids
            }
        })

    # маршруты
    for route in routes:
        path = route['path']
        coords = []
        for k in range(len(path)-1):
            u, v = path[k], path[k+1]
            d = G[u][v]
            if 'geometry' in d and d['geometry']:
                ec = d['geometry']
            else:
                ec = [(G.nodes[u]['x'], G.nodes[u]['y']),
                      (G.nodes[v]['x'], G.nodes[v]['y'])]
            if not coords:
                coords.extend(ec)
            else:
                coords.extend(ec[1:])
        features_out.append({
            'type': 'Feature',
            'geometry': {'type': 'LineString',
                         'coordinates': [[c[0], c[1]] for c in coords]},
            'properties': {
                'type': 'route',
                'from': route['from'], 'to': route['to'],
                'length_m': round(route['length_m'], 2)
            }
        })

    # рёбра дорожного графа
    for u, v, d in G.edges(data=True):
        if d.get('road_type') == 'station_link':
            continue
        if 'geometry' in d and d['geometry']:
            coords = [[lon, lat] for lon, lat in d['geometry']]
        else:
            coords = [
                [G.nodes[u]['x'], G.nodes[u]['y']],
                [G.nodes[v]['x'], G.nodes[v]['y']]
            ]
        features_out.append({
            'type': 'Feature',
            'geometry': {'type': 'LineString', 'coordinates': coords},
            'properties': {
                'type': 'road',
                'source': u, 'target': v,
                'length_m': round(d.get('length_m', 0), 2),
                'road_type': d.get('road_type', '')
            }
        })

    bbox_meta = {}
    if BBOX is not None:
        bbox_meta = {'bbox': {'N': BBOX_N, 'S': BBOX_S, 'W': BBOX_W, 'E': BBOX_E}}

    geojson = {
        'type': 'FeatureCollection',
        'metadata': {
            'total_stations': len(stations),
            'selected_stations': n_sel,
            'routes_count': len(routes),
            'min_route_km': MIN_ROUTE_KM,
            'max_route_km': MAX_ROUTE_KM,
            'graph_nodes': G.number_of_nodes(),
            'graph_edges': G.number_of_edges(),
            'graph_components': nx.number_connected_components(G),
            **bbox_meta
        },
        'features': features_out
    }

    gj_path = os.path.join(out_dir, f'roads_and_tracks_{ts}.geojson')
    with open(gj_path, 'w', encoding='utf-8') as f:
        json.dump(geojson, f, ensure_ascii=False)
    sz = os.path.getsize(gj_path) / (1024 * 1024)
    print(f"  GeoJSON: {gj_path}  ({sz:.1f} МБ)")
    print(f"\nСохранено в: {out_dir}")
else:
    print("Сохранение пропущено.")